Check the GPU, make sure it is Tesla P100-PCIE or V100-SXM2

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required libs

### please update Albumentations to version>=0.3.0 for `Lambda` transform support
#!pip install -U albumentations>=0.3.0 --user 
!pip install opencv-python-headless==4.5.4.60
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install -U --pre segmentation-models --user
# https://github.com/keras-team/keras/issues/14265#issuecomment-720376773
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.6 MB 85.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-pl9uwtqj
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-pl9uwtqj
  Created wheel for albumentations: filename=albumentations-1.2.0-py3-none-any.whl size=113560 sha256=fae041049926f3c6e559e2f5de9af79072fa51fddef85ce75a16573d117e508c
  Stored in directory: /tmp/pip-ephem-wheel-cache-vdp4ovor/wheels/63/11/1a/c77caf3ae9b9b6d57b3ee5e6a41a50f3bc12c66a70f6b90bf0
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
Looking in indexes: https://pypi.org/simple, https:

# Restart Runtime then execute the following cells

##Import libraries
#### Reqirements
- keras >= 2.2.0 or tensorflow >= 1.13
- segmenation-models==1.0.*
- albumentations==0.3.0

In [ ]:
%tensorflow_version 1.x
# 1.15.2 / 2.3.1
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
print(tf.test.gpu_device_name())

TensorFlow 1.x selected.
1.15.2
2.3.1



Using TensorFlow backend.


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A
import segmentation_models as sm

from PIL import Image
from skimage import io
import random as rng
# classification
from tensorflow.keras.models import Model
import scipy as sp
from google.colab.patches import cv2_imshow

Segmentation Models: using `keras` framework.


# Segmentation model

In [ ]:
BACKBONE = "mobilenetv2" # 'efficientnetb1' 'mobilenetv2'
CLASSES = ['bg', 'heart']
preprocess_input = sm.get_preprocessing(BACKBONE)
n_classes = len(CLASSES) + 1  
activation = 'softmax'
# # create model
# # model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
# model = keras.models.load_model('drive/MyDrive/colab_data/chimei_2_views/checkpoints/UNet/UNET_mobilenetv2_no_weight_model.h5')
# # load weights
# model.load_weights('drive/MyDrive/colab_data/chimei_2_views/checkpoints/UNet/20220413_568_MobileNetV2/UNET_568_1to100_jf_100_0.91_weight.h5') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/tensorflow-1.15.2/python3.7/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## h5 to TensorFlow SavedModel Format
https://ksingh7.medium.com/part-iv-convert-keras-model-to-tensorflow-savedmodel-format-205a168c0eef

In [ ]:
model = tf.keras.models.load_model('drive/MyDrive/colab_data/chimei_2_views/checkpoints/UNet/UNET_mobilenetv2_no_weight_model.h5')
model.load_weights('drive/MyDrive/colab_data/chimei_2_views/checkpoints/UNet/20220413_568_MobileNetV2/UNET_568_1to100_jf_100_0.91_weight.h5') 

In [ ]:
tf_savedmodel_folder = 'drive/MyDrive/colab_data/chimei_2_views/checkpoints/UNet/20220413_568_MobileNetV2/UNET_568_1to100_jf_100_0.91_tfsavedmodel'
tf.saved_model.save(model, tf_savedmodel_folder)
print("successfully saved keras model file to tensorflow model file")

In [ ]:
model = tf.keras.models.load_model(tf_savedmodel_folder)

# AP view classification model

In [ ]:
model_AP = tf.keras.models.load_model('drive/MyDrive/colab_data/chimei_2_views/checkpoints/Classification/20211213_366_custom_AP/custom_366_4_classes_96_0.99.h5')
gap_weights_AP = model_AP.layers[-1].get_weights()[0]
cam_model_AP  = Model(inputs=model_AP.input, outputs=(model_AP.layers[-3].output, model_AP.layers[-1].output))
cam_model_AP.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_______________________

## h5 to TensorFlow SavedModel Format

In [ ]:
tf_savedmodel_folder_AP = 'drive/MyDrive/colab_data/chimei_2_views/checkpoints/Classification/20211213_366_custom_AP/custom_366_4_classes_96_0.99_tfsavedmodel'
tf.saved_model.save(model_AP, tf_savedmodel_folder_AP)
print("successfully saved keras model file to tensorflow model file")

INFO:tensorflow:Assets written to: custom_366_4_classes_96_0.99_tfsavedmodel/assets
successfully saved keras model file to tensorflow model file


In [ ]:
model_AP = tf.keras.models.load_model(tf_savedmodel_folder_AP)
gap_weights_AP = model_AP.layers[-1].get_weights()[0]
cam_model_AP  = Model(inputs=model_AP.input, outputs=(model_AP.layers[-3].output, model_AP.layers[-1].output))
cam_model_AP.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0   

# Lateral view classification model

In [ ]:
model_Lateral = tf.keras.models.load_model('drive/MyDrive/colab_data/chimei_2_views/checkpoints/Classification/20211209_147_custom_lateral/custom_147_3_classes_53_0.95.h5')
gap_weights_Lateral = model_Lateral.layers[-1].get_weights()[0]
cam_model_Lateral  = Model(inputs=model_Lateral.input, outputs=(model_Lateral.layers[-3].output, model_Lateral.layers[-1].output))
cam_model_Lateral.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0   

## h5 to TensorFlow SavedModel Format

In [ ]:
tf_savedmodel_folder_Lateral = 'drive/MyDrive/colab_data/chimei_2_views/checkpoints/Classification/20211209_147_custom_lateral/custom_147_3_classes_53_0.95_tfsavedmodel'
tf.saved_model.save(model_Lateral, tf_savedmodel_folder_Lateral)
print("successfully saved keras model file to tensorflow model file")

INFO:tensorflow:Assets written to: custom_147_3_classes_53_0.95_tfsavedmodel/assets
successfully saved keras model file to tensorflow model file


In [ ]:
model_Lateral = tf.keras.models.load_model(tf_savedmodel_folder_Lateral)
gap_weights_Lateral = model_Lateral.layers[-1].get_weights()[0]
cam_model_Lateral  = Model(inputs=model_Lateral.input, outputs=(model_Lateral.layers[-3].output, model_Lateral.layers[-1].output))
cam_model_Lateral.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0   